In [26]:
!git clone https://github.com/trduc97/neural_medical_qa.git
    
%cd neural_medical_qa
!pip install -r requirements.txt

Cloning into 'neural_medical_qa'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 69 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (69/69), 1.74 MiB | 3.36 MiB/s, done.
/kaggle/working/neural_medical_qa/neural_medical_qa


In [ ]:
!pip install transformers

# 0. Dataset

In [2]:
from import_datasets import load_bioasq_pubmedqa,  train_val_test_split

bioasq, pubmedqa = load_bioasq_pubmedqa()

# Display the first few samples of the PubMedQA dataset
print(pubmedqa['train'].to_pandas().head())

responses = pubmedqa['train']['final_decision']
# Counting the occurrences of each value
yes_count = responses.count('yes')
no_count = responses.count('no')
maybe_count = responses.count('maybe')

# Display the counts
print(f"Yes: {yes_count}")
print(f"No: {no_count}")
print(f"Maybe: {maybe_count}")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

      pubid                                           question  \
0  21645374  Do mitochondria play a role in remodelling lac...   
1  16418930  Landolt C and snellen e acuity: differences in...   
2   9488747  Syncope during bathing in infants, a pediatric...   
3  17208539  Are the long-term results of the transanal pul...   
4  10808977  Can tailored interventions increase mammograph...   

                                             context  \
0  {'contexts': ['Programmed cell death (PCD) is ...   
1  {'contexts': ['Assessment of visual acuity dep...   
2  {'contexts': ['Apparent life-threatening event...   
3  {'contexts': ['The transanal endorectal pull-t...   
4  {'contexts': ['Telephone counseling and tailor...   

                                         long_answer final_decision  \
0  Results depicted mitochondrial dynamics in viv...            yes   
1  Using the charts described, there was only a s...             no   
2  "Aquagenic maladies" could be a pediatric form... 

# 1. Split the dataset

In [3]:
pubmedqa_train,pubmedqa_val, pubmedqa_test = train_val_test_split(pubmedqa)
print(f"Train size: {len(pubmedqa_train)}")
print(f"Validation size: {len(pubmedqa_val)}")
print(f"Test size: {len(pubmedqa_test)}")

Train size: 750
Validation size: 100
Test size: 150


# 2. EDA

In [5]:
from collections import defaultdict
# Initialize a defaultdict to hold the bucket counts
length_buckets = defaultdict(int)

# Define the bucket size
bucket_size = 128

# Loop through each string in the list
for s in pubmedqa_train['long_answer']:
    # Determine the bucket for the current string length
    bucket = (len(s) // bucket_size) * bucket_size
    # Increment the count for the appropriate bucket
    length_buckets[bucket] += 1

# Display the counts for each bucket
for bucket, count in sorted(length_buckets.items()):
    print(f"Length {bucket} - {bucket + bucket_size - 1}: {count} strings")

Length 0 - 127: 66 strings
Length 128 - 255: 326 strings
Length 256 - 383: 244 strings
Length 384 - 511: 82 strings
Length 512 - 639: 25 strings
Length 640 - 767: 4 strings
Length 768 - 895: 3 strings


# 3. Model

In [18]:
# List all available devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

results={}

## 3.1. Text representation

### 3.1.1. TF-IDF

### 3.1.2. Embedding layer

### 3.1.3. ELMo 

### 3.1.4. BERT (base-uncased)

In [ ]:
# List all available devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

embedding='bert'
model='dense'
model_name=embedding+'_'+model

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['question'], examples['long_answer'], truncation=True, padding='max_length', max_length=512)

# Tokenize the datasets
train_dataset = pubmedqa_train.map(preprocess_function, batched=True)
val_dataset = pubmedqa_val.map(preprocess_function, batched=True)
test_dataset = pubmedqa_test.map(preprocess_function, batched=True)

# Convert the datasets to TensorFlow datasets
def convert_to_tf_dataset(dataset, shuffle=True, batch_size=32):
    def gen():
        for ex in dataset:
            yield ({'input_ids': ex['input_ids'], 'attention_mask': ex['attention_mask']}, ex['decision_encoded'])
    
    return tf.data.Dataset.from_generator(gen, 
                                          ({'input_ids': tf.int32, 'attention_mask': tf.int32}, tf.int64),
                                          ({'input_ids': tf.TensorShape([None]), 'attention_mask': tf.TensorShape([None])}, tf.TensorShape([]))
                                         ).shuffle(1000).batch(batch_size)

train_tf_dataset = convert_to_tf_dataset(train_dataset)
val_tf_dataset = convert_to_tf_dataset(val_dataset, shuffle=False)
test_tf_dataset = convert_to_tf_dataset(test_dataset, shuffle=False)

class PubMedQAModel(tf.keras.Model):
    def __init__(self, model_name='bert-base-uncased',classes=3, dropout=0.3,nodes=32):
        super(PubMedQAModel, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name)      
        # Freeze all layers except the last two
        for layer in self.bert.layers:
            layer.trainable = False
        for layer in self.bert.bert.encoder.layer[-2:]:
            layer.trainable = True
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)
        self.dense1 = tf.keras.layers.Dense(nodes, activation='relu')
        self.classifier = tf.keras.layers.Dense(classes, activation='softmax')
    
    def call(self, inputs):
        input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropout_output1 = self.dropout1(pooled_output, training=False)
        dense_output1= self.dense1(dropout_output1)
        dropout_output2 = self.dropout2(dense_output1, training=False)        
        logits = self.classifier(dropout_output2)
        return logits

model = PubMedQAModel()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# Train the model
epochs = 10

history = model.fit(train_tf_dataset, validation_data=val_tf_dataset, epochs=epochs)

# Evaluate the model
loss, accuracy = model.evaluate(test_tf_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


### 3.1.5. PubMedBert

## 3.2. Models 

### 3.2.1. Match-LSTM

### 3.2.2. BiDAF

# 4. Evaluate the results